# Post Office Simulation

## Problem Statement

A post office has a window with one worker. The window is (open/busy/closed)

The window opens and closes  several times through the day with a pre-ahead schedule to makethe worker work in internal office work.

If the worker is serving one customer then the window is busy till he finishes.  

If a customer come during the window is closed or busy , then he waits in the queue.

The goal is to optimize the open/close schedule (time and duration).

Model the behavior of this system.  

Report the average waiting time in the system.  

Make any reasonable assumptions if needed.


We will do some needed imports

In [1]:
import heapq
import numpy as np
import collections

We define the state class which is responsible for handling all entities states

In [2]:
MAX_TIME = 10000
DELAY = 0
class State: # the state of the post office window
    def __init__(self): # initialization function
        self.window_state = 0 # 0 for closed, 1 for open, 2 for busy
        self.last_event_time = 0
        self.current_event_time = 0
        self.window_close_time = MAX_TIME
        self.current_job_finish_time = MAX_TIME
        self.waiting_queue = collections.deque()
        self.waiting_customers = 0
        self.total_customers_served = 0
        self.total_waiting_time = 0
        
    # Entity-related states
    def current_window_state(self): # get current state of the window
        return self.window_state
    def add_customer(self, customer): # add a customer to the queue
        self.waiting_queue.append(customer)
        self.increment_total_customers_served()
    def remove_customer(self): # remove the customer from the waiting queue and calculate his waiting time
        x = self.waiting_queue.popleft()
        self.update_total_waiting_time(x[0], x[1] + self.get_current_event_time()) # x[0] is the time of arrival of the customer
    def close_window(self): # make the window closed
        self.window_state = 0
    def open_window(self): # make the window opened
        self.window_state = 1
    def busy_window(self): # make the window busy
        self.window_state = 2

    # Needed Time Parameters
    def update_time_parameters(self, time):
        self.last_event_time = self.current_event_time
        self.current_event_time = time
    def get_last_event_time(self):
        return self.last_event_time
    def get_current_event_time(self):
        return self.current_event_time
    
    def set_window_close_time(self, time):
        self.window_close_time = time
    def get_window_close_time(self):
        return self.window_close_time

    def set_current_job_finish_time(self, time):
        self.current_job_finish_time = time
    def get_current_job_finish_time(self):
        return self.current_job_finish_time
 
    # Statistics Collection
    def update_total_waiting_time(self, time1, time3):
        self.total_waiting_time += (time3 - time1)
    def get_total_waiting_time(self):
        return self.total_waiting_time
    
    def increment_total_customers_served(self):
        self.total_customers_served += 1
    def get_total_customers_served(self):
        return self.total_customers_served
    def get_waiting_customers(self):
        self.waiting_customers = len(self.waiting_queue)
        return self.waiting_customers
    def __str__(self):
        print("")

Now, we define the Event class, the parent class of all events

In [3]:
class Event:
    def start_time(self):
        return self.t1
    def service_time(self):
        return self.t2
    def __str__(self):
        return self.name + "(" + str(self.t1) + ", " + str(self.t2) + ")"
    def __lt__(self, other):
        return self.t1 < other.t1

The customer arrival event is defined below  
For a customer, t1 defines the arrival time, and t2 defines the time he takes to be served

Note: For any event, t1 represents the time at which the event starts, and t2 represents the duration which it lasts  


In [4]:
class CUSTOMER(Event):
    def __init__(self, time1, time2):
        self.t1 = time1
        self.t2 = time2
        self.name = "CUSTOMER"
    def action(self, queue, state):
        state.update_time_parameters(self.t1)
        if state.current_window_state() == 1: # if a customer arrives and the window is opened, then serve the customer immediately; for modularity and generality,
            #  I made it like he goes to the wait queue and waits for zero additional time
            queue.insert(SERVE(self.t1 + DELAY, self.t2)) 
        else: # if the window is closed or busy, add the customer to the waiting list
            state.add_customer((self.t1, self.t2))

The Window Opening class, inheriting from Event class  
For a window, t1 defines the time from which it is opened, and t2 defines the times for which it is open

In [5]:
class WOPEN(Event): # Window opening event
    def __init__(self, time1, time2):
        self.t1 = time1
        self.t2 = time2
        self.name = "W_OPEN"
    def action(self, queue, state):
        state.set_window_close_time(self.t1 + self.t2)
        state.update_time_parameters(self.t1) # I believe that, ideally, this shouldn't be done here; it should be maintained by EVENT Queue itself
        queue.insert(WCLOSE(self.t1 + self.t2)) # set the window closing time
        if state.current_window_state() == 0: # if the window is closed, open it and schedule the customers 
            state.open_window()
            if(state.get_waiting_customers() > 0 and  state.waiting_queue[0][0] <= self.t1):
                queue.insert(SERVE(self.t1 + DELAY, state.waiting_queue[0][1]))
        elif state.current_window_state() == 2: # window is busy
            # I assumed that an open event doesn't invoke an action on a window that is closed or busy
            # If this is not the case in your assumption, then it is here the place where you should account for your assunmption
            pass
        else: # window is open already
            pass

The SERVE class, inherits from Event. It represents the internal event of serving a customer

In [6]:
class SERVE(Event):
    def __init__(self, time1, time2):
        self.t1 = time1
        self.t2 = time2
        self.name = "SERVE"
    def action(self, queue, state):
        state.set_current_job_finish_time(self.t1 + self.t2)
        state.update_time_parameters(self.t1)
        state.remove_customer() # remove the customer from the queue and take him in, making the window busy
        state.busy_window()
        queue.insert(WFREE(self.t1 + self.t2, state.get_window_close_time() - (self.t1 + self.t2)))

The window-closing event
We don't know for how long the window is to be closed, so we assume it will be closed for an arbitrarily long amount of time MAX_TIME

In [7]:
class WCLOSE(Event):
    def __init__(self, time1):
        self.t1 = time1
        self.t2 = MAX_TIME
        self.name = "W_CLOSE"
    def action(self, queue, state):
        if state.current_window_state() == 2: # if the window is currently busy serving a customer, then continue to serve him until he is finished
            closing_time = state.get_current_job_finish_time()
            queue.insert(WCLOSE(closing_time))
        else: # if no customer is being served, just close the window
            state.close_window()
    def __str__(self): # This overrides the function inherited from the EVENT class
        return self.name + "(" + str(self.t1) + ")"

The Free Window Event is shown.  
This event represents the freeing of a window after a customer has been served.  
This, in essence, is an internal event to open the window.
  
Why didn't we use the WOPEN event?  
1- To signify the distiniction between the window opening due to an external event, and it being freed after having served a customer; opening due to an internal event.  
2- For better maintainability and extensibility of the model later.

The drawback, for the current time at least, is obviously that we added an additional event to our model.  
At a first glance, one might think that the fewer events you have (i.e the lesser the number of transitions in the system), the simpler the model is.  
This is indeed a valid thought; representing our system with a model with the fewest possible transitions, events and states should always be a goal.  
Here, however, the model is not so complex, and the benefits mentioned above are worthy of that split.

In [8]:
class WFREE(Event):
    def __init__(self, time1, time2):
        self.t1 = time1
        self.t2 = time2
        self.name = "W_FREE"
    def action(self, queue, state):
        self.t2 = state.get_window_close_time()
        state.open_window()

        window_reopen_time = self.t1 # the time by which the window is supposed to reopen after serving the current customer; the time at which the window is free again

        window_close_time = state.get_window_close_time() # the time at which the current window is supposed to close
        # print("window_close_time = ", window_close_time, ", window_reopen_time = ", window_reopen_time)

        next_window_opening_duration = window_close_time - window_reopen_time

        if(next_window_opening_duration > 0):
            if(state.get_waiting_customers() > 0 and  state.waiting_queue[0][0] <= self.t1):
                queue.insert(SERVE(window_reopen_time + DELAY, state.waiting_queue[0][1]))
            # queue.insert(WOPEN(window_reopen_time, next_window_opening_duration))
            state.set_window_close_time(window_reopen_time + next_window_opening_duration)
        else:
            queue.insert(WCLOSE(window_reopen_time))
            



The EventQueue is defined

In [9]:
class EventQueue:
    def __init__(self):
        self.q = []
    def notempty(self):
        return len(self.q) > 0
    def remaining(self):
        return len(self.q)
    def insert(self, event):
        heapq.heappush(self.q, event)
    def next(self):
        return heapq.heappop(self.q)

The main is shown below with a sample run

In [10]:
### MAIN

Q = EventQueue()

Q.insert( CUSTOMER(0,2) ) #a customer arrives at t=0 , and requires 2 t to finish his processing.
Q.insert( CUSTOMER(1,1) ) 
Q.insert( CUSTOMER(1.1,3) ) 
Q.insert( WOPEN(2,5) )  # the window opens at 2 and closes after 5 t i.e closes at 7 or if it processes a customer then after the customer finishes.
Q.insert( CUSTOMER(2,1) ) 
Q.insert( CUSTOMER(3,1) ) 
Q.insert( WOPEN(10,5) ) 
    
S = State()

# Processing events until the queue is Q is empty
while Q.notempty():
    e = Q.next()
    print( e )
    e.action(Q,S)
    print(S.current_window_state())
total_wait = S.get_total_waiting_time()
total_customers = S.get_total_customers_served()
result = total_wait/total_customers
print("Average Waiting Time = %f" %result)

CUSTOMER(0, 2)
0
CUSTOMER(1, 1)
0
CUSTOMER(1.1, 3)
0
CUSTOMER(2, 1)
0
W_OPEN(2, 5)
1
SERVE(2, 2)
2
CUSTOMER(3, 1)
2
W_FREE(4, 3)
1
SERVE(4, 1)
2
W_FREE(5, 2)
1
SERVE(5, 3)
2
W_CLOSE(7)
2
W_FREE(8, -1)
1
W_CLOSE(8)
0
W_CLOSE(8)
0
W_OPEN(10, 5)
1
SERVE(10, 1)
2
W_FREE(11, 4)
1
SERVE(11, 1)
2
W_FREE(12, 3)
1
W_CLOSE(15)
0
Average Waiting Time = 6.580000
